In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import sys
import os
path = '/content/drive/My Drive'
sys.path.append(path)
os.chdir(path)
%cd BDA 21

/content/drive/My Drive/BDA 21


load_data

In [3]:
import pandas as pd


def load_dataset(path):
    data = pd.read_csv(path)
    y = data.loc[:, "label"].values.astype(int)
    X = data.iloc[:, 3:9].values
    return X, y

resample

In [4]:
!pip install pytorch-tabnet
!pip install imblearn

In [5]:
from imblearn.over_sampling import * 
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
import numpy as np


def resampling(X, y):
    kmeans_sm = KMeansSMOTE(random_state=42, cluster_balance_threshold=0.05)
    X_res, y_res = kmeans_sm.fit_resample(X, y)
    # svm_sm = SVMSMOTE(random_state=42)
    # X_res2, y_res2 = svm_sm.fit_resample(X, y)
    # return np.vstack((X_res, X_res2)), np.hstack((y_res, y_res2))
    return X_res, y_res


def data_preprocess(X):
    return normalize(X, axis=0)


def cross_validation(X_train, Y_train):
    five_fold_data = list()
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    for train_index, eval_index in kf.split(X_train):
        x_train, x_eval = X_train[train_index], X_train[eval_index]
        y_train, y_eval = Y_train[train_index], Y_train[eval_index]
        five_fold_data.append([(x_train, y_train), (x_eval, y_eval)])
    return five_fold_data

use tabnet with pretraining

In [85]:
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, confusion_matrix


def train(five_fold_data, X_pretrain):
    unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax' # "sparsemax"
    )

    unsupervised_model.fit(
        X_train=X_pretrain,
        eval_set=[X_pretrain],
        pretraining_ratio=0.8,
    )
    clf = TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2),
        scheduler_params={"step_size":10, # how to use learning rate scheduler
                        "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax' # This will be overwritten if using pretrain model
    )

    model_sets = list()
    # indices = np.random.choice(5, size=3, replace=False)
    for i, [(x_train, y_train), (x_eval, y_eval)] in enumerate(five_fold_data):
        clf.fit(
            X_train=x_train, y_train=y_train, 
            eval_set=[(x_train, y_train), (x_eval, y_eval)],
            eval_name=['train', 'valid'],
            eval_metric=['auc', 'accuracy'],
            from_unsupervised=unsupervised_model
    )
        # if i in indices:
        model_sets.append(clf)
    return model_sets


def tabnet_test(model_sets, X_test, y_test):
    results = np.zeros((5, len(X_test)))
    for i, model in enumerate(model_sets):
        preds = model.predict_proba(X_test)
        results[i] = preds[:, 1]
    best_roc_auc = 0.5
    best_factor = 0.85
    # for factor in np.arange(0.7, 0.95, 0.1):
    #     score = results.mean(axis=-2) * (0.5/factor)
    #     auc = roc_auc_score(y_test, score)
    #     if auc > best_roc_auc:
    #         best_roc_auc = auc 
    #         best_factor = factor
    score = results.mean(axis=-2) * (0.5/best_factor)
    auc = roc_auc_score(y_test, score)    
    pred = (score > 0.5).astype(int)
    recall = recall_score(y_true=y_test, y_pred=pred)
    acc = accuracy_score(y_pred=pred, y_true=y_test)
    return acc, auc, recall, score

In [75]:
from sklearn.ensemble import AdaBoostClassifier


def tree_decider(X_train, y_train, X_test, y_test):
    clf = AdaBoostClassifier(n_estimators=100, random_state=0)  
    clf.fit(X_train, y_train)
    best_roc_auc = 0.5
    best_factor = 0.5
    for factor in np.arange(0.25, 0.5, 0.1):
        prob = clf.predict_proba(X_test)[:, 1] * (0.5/best_factor)
        auc = roc_auc_score(y_test, prob, average="samples")
        if auc > best_roc_auc:
            best_roc_auc = auc 
            best_factor = factor
    prob = clf.predict_proba(X_test)[:, 1] * (0.5/best_factor)
    pred = np.where(prob > 0.5, 1, 0)
    acc = accuracy_score(y_pred=pred, y_true=y_test)
    auc = roc_auc_score(y_test, prob, average="samples")
    recall = recall_score(y_test, pred)
    return acc, auc, recall, prob

In [76]:
from sklearn.linear_model import LogisticRegressionCV


def linear_model(train_X, train_y, test_X, test_y):
    clf = LogisticRegressionCV(cv=5, penalty="l2", random_state=42, multi_class="ovr", solver="liblinear")
    clf.fit(train_X, train_y)
    best_roc_auc = 0.5
    best_factor = 0.5
    for factor in np.arange(0.25, 0.5, 0.1):
        prob = clf.predict_proba(test_X)[:, 1] * (0.5/factor)
        auc = roc_auc_score(test_y, prob, average="samples")
        if auc > best_roc_auc:
            best_roc_auc = auc 
            best_factor = factor
    prob = clf.predict_proba(test_X)[:, 1] * (0.5/best_factor)
    pred = np.where(prob > 0.5, 1, 0)
    acc = accuracy_score(y_pred=pred, y_true=test_y)
    auc = roc_auc_score(test_y, prob, average="samples")
    recall = recall_score(test_y, pred)
    return acc, auc, recall, prob

ensemble

In [103]:
def ensemble(results_tabnet, results_linear, results_tree, y_test):
    final_results = np.empty((len(results_tabnet)))
    score_avg = (results_tabnet + results_linear + results_tree) / 3
    for i in range(len(results_tabnet)):
        final_results[i] = (score_avg[i] > 0.5).astype(int)
    cf_matrix = confusion_matrix(y_test, final_results)
    return cf_matrix

In [78]:
def full_train():
    train_X, train_y = load_dataset("player_stats_2019_2020.csv")
    test_X, test_y = load_dataset("player_stats_2020_2021.csv")
    train_X_norm = data_preprocess(train_X)
    test_X_norm = data_preprocess(test_X)
    acc, auc, recall, results_linear = linear_model(train_X_norm, train_y, test_X_norm, test_y)
    print("linear model: acc is {:.5f}, auc is {:.5f} and recall is {:.5f}".format(acc, auc, recall))
    acc, auc, recall, results_tree = tree_decider(train_X_norm, train_y, test_X_norm, test_y)
    print("tree model: acc is {:.5f}, auc is {:.5f} and recall is {:.5f}".format(acc, auc, recall))
    
    train_X_re, train_y_re = resampling(train_X, train_y)

    five_fold_data = cross_validation(train_X_re, train_y_re)
    models = train(five_fold_data, train_X)
    acc_score, auc_score, the_recall_score, results_tabnet = tabnet_test(models, test_X, test_y)
    print("accuracy score is {:.5f}, roc auc score is {:.5f} and recall score is {:.5f}".format(acc_score, auc_score, the_recall_score))

    return results_tabnet, results_linear, results_tree, test_y

In [79]:
results_tabnet, results_linear, results_tree, test_y = full_train()

linear model: acc is 0.97222, auc is 0.97321 and recall is 0.46667
tree model: acc is 0.90000, auc is 0.93517 and recall is 0.86667
Device used : cpu
epoch 0  | loss: 7.01153 | val_0_unsup_loss: 431.34521|  0:00:00s
epoch 1  | loss: 5.52148 | val_0_unsup_loss: 126.34342|  0:00:00s
epoch 2  | loss: 3.85334 | val_0_unsup_loss: 48.94502|  0:00:00s
epoch 3  | loss: 3.32467 | val_0_unsup_loss: 25.81126|  0:00:00s
epoch 4  | loss: 2.9094  | val_0_unsup_loss: 23.02277|  0:00:00s
epoch 5  | loss: 2.75511 | val_0_unsup_loss: 14.01752|  0:00:00s
epoch 6  | loss: 2.79977 | val_0_unsup_loss: 7.89436 |  0:00:00s
epoch 7  | loss: 2.34031 | val_0_unsup_loss: 5.02979 |  0:00:00s
epoch 8  | loss: 2.31576 | val_0_unsup_loss: 5.18786 |  0:00:00s
epoch 9  | loss: 2.01735 | val_0_unsup_loss: 6.05944 |  0:00:00s
epoch 10 | loss: 1.97868 | val_0_unsup_loss: 6.39478 |  0:00:00s
epoch 11 | loss: 1.91815 | val_0_unsup_loss: 6.39032 |  0:00:00s
epoch 12 | loss: 1.83278 | val_0_unsup_loss: 6.82424 |  0:00:00s
epo

In [104]:
cf_matrix = ensemble(results_tabnet, results_linear, results_tree, test_y)
print("final confusion matrix is:")
print(cf_matrix)

final confusion matrix is:
[[506  19]
 [  3  12]]
